<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/emotion_GPT2_as_text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

In [1]:
# 필수 라이브러리 임포트
import json             # JSON 데이터 구문 분석을 위한 라이브러리
import random          # 시드 설정 및 데이터 셔플링을 위한 라이브러리
import gzip            # 데이터셋 압축 해제를 위한 라이브러리
import requests        # URL에서 데이터셋 다운로드를 위한 라이브러리
import torch           # 메인 파이토치 라이브러리
from torch.utils.data import Dataset, DataLoader  # 데이터셋 처리를 위한 라이브러리
from transformers import AutoTokenizer, AutoModelForCausalLM  # 허깅 페이스 모델 구성 요소
from torch.optim import AdamW    # 훈련을 위한 옵티마이저
from tqdm import tqdm   # 진행률 표시줄 유틸리티
import re              # 텍스트 정규화를 위한 라이브러리

def set_seed(seed):
    """
    다양한 라이브러리에서 재현성을 위해 난수 시드를 설정합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값
    """
    # 파이썬 내장 random 시드 설정
    random.seed(seed)
    # 파이토치 CPU 난수 시드 설정
    torch.manual_seed(seed)
    # 사용 가능한 모든 GPU에 대한 시드 설정
    torch.cuda.manual_seed_all(seed)
    # cuDNN이 결정론적 알고리즘을 사용하도록 요청
    torch.backends.cudnn.deterministic = True
    # 일관된 동작을 위해 cuDNN의 자동 튜너 비활성화
    torch.backends.cudnn.benchmark = False

def build_prompt(text):
    """
    감정 분류를 위한 표준화된 프롬프트를 생성합니다.
    Args:
        text (str): 분류할 입력 텍스트
    Returns:
        str: 모델을 위한 형식화된 프롬프트
    """
    # 입력 텍스트를 일관된 프롬프트 구조로 형식화
    # 명시적인 작업 지시문과 예상 출력 형식 포함
    return f"Predict the emotion for the following text: {text}\nEmotion:"

def encode_text(tokenizer, text, return_tensor=False):
    """
    제공된 토크나이저를 사용하여 텍스트를 인코딩합니다.
    Args:
        tokenizer: 허깅 페이스 토크나이저
        text (str): 인코딩할 텍스트
        return_tensor (bool): 파이토치 텐서를 반환할지 여부
    Returns:
        토큰 ID의 목록 또는 텐서
    """
    # 텐서 출력이 요청된 경우, 파이토치 텐서로 인코딩
    if return_tensor:
        return tokenizer.encode(
            text, add_special_tokens=False, return_tensors="pt"
        )
    # 그렇지 않으면 토큰 ID 목록 반환
    else:
        return tokenizer.encode(text, add_special_tokens=False)

def decode_text(tokenizer, token_ids):
    """
    토큰 ID를 텍스트로 다시 디코딩합니다.
    Args:
        tokenizer: 허깅 페이스 토크나이저
        token_ids: 토큰 ID의 목록 또는 텐서
    Returns:
        str: 디코딩된 텍스트
    """
    # 특수 토큰을 건너뛰고 토큰 ID를 텍스트로 다시 변환
    return tokenizer.decode(token_ids, skip_special_tokens=True)

class PromptCompletionDataset(Dataset):
    """
    프롬프트-완성 쌍을 위한 파이토치 데이터셋.
    텍스트 데이터를 모델 준비 형식으로 변환하는 작업을 처리합니다.
    Args:
        data (list): 프롬프트와 완성을 포함하는 사전 목록
        tokenizer: 허깅 페이스 토크나이저
    """
    def __init__(self, data, tokenizer):
        # 나중에 사용하기 위해 원시 데이터와 토크나이저 저장
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        # 데이터셋의 총 샘플 수 반환
        return len(self.data)

    def __getitem__(self, idx):
        """
        단일 훈련 샘플을 반환합니다.
        Args:
            idx (int): 가져올 샘플의 인덱스
        Returns:
            dict: input_ids, labels, prompt, expected_completion을 포함
        """
        # 데이터셋에서 특정 샘플 가져오기
        item = self.data[idx]
        prompt = item["prompt"]
        completion = item["completion"]
        # 프롬프트와 완성 모두에 대해 텍스트를 토큰 ID로 변환
        encoded_prompt = encode_text(self.tokenizer, prompt)
        encoded_completion = encode_text(self.tokenizer, completion)
        # 시퀀스 끝 토큰 ID 가져오기
        eos_token = self.tokenizer.eos_token_id
        # EOS 토큰과 함께 프롬프트와 완성 토큰 결합
        input_ids = encoded_prompt + encoded_completion + [eos_token]
        # 레이블 생성: 프롬프트의 경우 -100(손실에서 무시됨), 학습을 위한 완성 토큰
        labels = [-100] * len(encoded_prompt) + encoded_completion + [eos_token]
        return {
            "input_ids": input_ids,
            "labels": labels,
            "prompt": prompt,
            "expected_completion": completion
        }

def collate_fn(batch):
    """
    샘플 배치를 훈련 준비 형식으로 통합합니다.
    패딩과 텐서 변환을 처리합니다.
    Args:
        batch: 데이터셋의 샘플 목록
    Returns:
        tuple: (input_ids, attention_mask, labels, prompts, expected_completions)
    """
    # 패딩을 위해 배치에서 가장 긴 시퀀스 찾기
    max_length = max(len(item["input_ids"]) for item in batch)
    # 패딩 토큰으로 입력 시퀀스를 max_length로 패딩
    input_ids = [
        item["input_ids"] +
        [tokenizer.pad_token_id] * (max_length - len(item["input_ids"]))
        for item in batch
    ]
    # -100으로 레이블 시퀀스 패딩(손실 계산에서 무시됨)
    labels = [
        item["labels"] +
        [-100] * (max_length - len(item["labels"]))
        for item in batch
    ]
    # 어텐션 마스크 생성: 실제 토큰은 1, 패딩은 0
    attention_mask = [
        [1] * len(item["input_ids"]) +
        [0] * (max_length - len(item["input_ids"]))
        for item in batch
    ]
    # 평가를 위해 원본 프롬프트와 완성 유지
    prompts = [item["prompt"] for item in batch]
    expected_completions = [item["expected_completion"] for item in batch]
    # 텍스트를 제외한 모든 항목을 파이토치 텐서로 변환
    return (
        torch.tensor(input_ids),
        torch.tensor(attention_mask),
        torch.tensor(labels),
        prompts,
        expected_completions
    )

def normalize_text(text):
    """
    일관된 비교를 위해 텍스트를 정규화합니다.
    Args:
        text (str): 입력 텍스트
    Returns:
        str: 정규화된 텍스트
    """
    # 선행/후행 공백 제거 및 소문자로 변환
    text = text.strip().lower()
    # 여러 공백 문자를 단일 공백으로 대체
    text = re.sub(r'\s+', ' ', text)
    return text

def calculate_accuracy(model, tokenizer, loader):
    """
    데이터셋에서 예측 정확도를 계산합니다.
    Args:
        model: 미세 튜닝된 모델
        tokenizer: 연관된 토크나이저
        loader: 평가 샘플을 포함하는 DataLoader
    Returns:
        float: 정확도 점수
    """
    # 모델을 평가 모드로 설정(드롭아웃 등 비활성화)
    model.eval()
    # 정확도 계산을 위한 카운터 초기화
    correct = 0
    total = 0
    # 효율성을 위해 그레이디언트 계산 비활성화
    with torch.no_grad():
        # 배치 반복
        for input_ids, attention_mask, labels, prompts, expected_completions in loader:
            # 배치의 각 샘플 처리
            for prompt, expected_completion in zip(prompts, expected_completions):
                # 이 프롬프트에 대한 모델 예측 생성
                generated_text = generate_text(model, tokenizer, prompt)
                # 예측과 예상 완성의 정규화된 버전 비교
                if normalize_text(generated_text) == normalize_text(expected_completion):
                    correct += 1
                total += 1
    # 정확도 계산, 빈 데이터셋 경우 처리
    accuracy = correct / total if total > 0 else 0
    # 모델을 훈련 모드로 재설정
    model.train()
    return accuracy

def generate_text(model, tokenizer, prompt, max_new_tokens=50):
    """
    주어진 프롬프트에 대한 텍스트 완성을 생성합니다.
    Args:
        model: 미세 튜닝된 모델
        tokenizer: 연관된 토크나이저
        prompt (str): 입력 프롬프트
        max_new_tokens (int): 생성할 최대 토큰 수
    Returns:
        str: 생성된 완성
    """
    # 프롬프트 인코딩 및 모델 장치로 이동
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    # 모델을 사용하여 완성 생성
    output_ids = model.generate(
        input_ids=input_ids["input_ids"],
        attention_mask=input_ids["attention_mask"],
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,        # 더 빠른 생성을 위해 KV 캐시 사용
        num_beams=1,           # 탐욕적 디코딩 사용
        do_sample=False,       # 샘플링 사용 안 함
    )[0]
    # 생성된 부분만 추출 및 디코딩(프롬프트 제외)
    generated_text = decode_text(tokenizer, output_ids[input_ids["input_ids"].shape[1]:])
    return generated_text.strip()

def test_model(model_path, test_input):
    """
    저장된 모델을 단일 입력으로 테스트합니다.
    Args:
        model_path (str): 저장된 모델의 경로
        test_input (str): 분류할 텍스트
    """
    # 장치 결정(사용 가능한 경우 GPU, 그렇지 않으면 CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 장치: {device}")
    # 저장된 모델 로드 및 적절한 장치로 이동
    model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    # 모델에 적절한 패딩 토큰 구성이 있는지 확인
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    # 프롬프트 생성 및 예측 생성
    prompt = build_prompt(test_input)
    generated_text = generate_text(model, tokenizer, prompt)
    # 결과 표시
    print(f"입력: {test_input}")
    print(f"생성된 감정: {generated_text}")

def download_and_prepare_data(data_url, tokenizer, batch_size, test_ratio=0.1):
    """
    훈련을 위해 데이터셋을 다운로드하고 준비합니다.
    Args:
        data_url (str): 데이터셋의 URL
        tokenizer: 텍스트 처리를 위한 토크나이저
        batch_size (int): DataLoader의 배치 크기
        test_ratio (float): 테스트를 위한 데이터 비율
    Returns:
        tuple: (train_loader, test_loader)
    """
    # 압축된 데이터셋 다운로드
    response = requests.get(data_url)
    # 콘텐츠 압축 해제 및 디코딩
    content = gzip.decompress(response.content).decode()
    # 각 라인을 JSON으로 구문 분석하고 프롬프트-완성 쌍으로 형식화
    dataset = []
    for entry in map(json.loads, content.splitlines()):
        dataset.append({
            "prompt": build_prompt(entry['text']),
            "completion": entry["label"].strip()
        })
    # 더 나은 분할을 위해 데이터셋 무작위 셔플
    random.shuffle(dataset)
    # 테스트 비율을 기반으로 분할 인덱스 계산
    split_index = int(len(dataset) * (1 - test_ratio))
    # 훈련 및 테스트 세트로 분할
    train_data = dataset[:split_index]
    test_data = dataset[split_index:]
    # 데이터셋 객체 생성
    train_dataset = PromptCompletionDataset(train_data, tokenizer)
    test_dataset = PromptCompletionDataset(test_data, tokenizer)
    # 적절한 설정으로 데이터 로더 생성
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,         # 훈련 데이터 셔플
        collate_fn=collate_fn  # 패딩을 위한 사용자 정의 통합 함수
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,        # 테스트 데이터는 셔플하지 않음
        collate_fn=collate_fn
    )
    return train_loader, test_loader

def get_hyperparameters():
    """
    훈련 하이퍼파라미터를 반환합니다.
    Returns:
        tuple: (num_epochs, batch_size, learning_rate)
    """
    # 학습과 효율성의 균형으로 2 에포크 훈련
    num_epochs = 2
    # 대부분의 GPU 메모리 크기에서 잘 작동하는 16의 배치 크기
    batch_size = 16
    # 트랜스포머 미세 튜닝을 위한 표준 학습률
    learning_rate = 5e-5
    return num_epochs, batch_size, learning_rate

In [2]:
# 재현성을 위해 난수 시드 설정
set_seed(42)

# 기본 훈련 매개변수 구성
data_url = "https://www.thelmbook.com/data/emotions"
model_name = "openai-community/gpt2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")

# 토크나이저 초기화 및 패딩 토큰 구성
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 사전 훈련된 모델 로드 및 적절한 장치로 이동
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# 훈련 하이퍼파라미터 가져오기
num_epochs, batch_size, learning_rate = get_hyperparameters()

# 훈련 데이터 로드 및 준비
train_loader, test_loader = download_and_prepare_data(data_url, tokenizer, batch_size)

# 학습률로 옵티마이저 초기화
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 훈련 루프
for epoch in range(num_epochs):
    # 에포크 메트릭 초기화
    total_loss = 0
    num_batches = 0

    # 이 에포크에 대한 진행률 표시줄 생성
    progress_bar = tqdm(train_loader, desc=f"에포크 {epoch+1}/{num_epochs}")

    # 각 배치 처리
    for input_ids, attention_mask, labels, _, _ in progress_bar:
        # 배치 데이터를 적절한 장치로 이동
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 손실 계산을 포함한 포워드 패스
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

        # 백워드 패스 및 최적화
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 진행 메트릭 업데이트
        total_loss += loss.item()
        num_batches += 1

        # 현재 손실로 진행률 표시줄 업데이트
        progress_bar.set_postfix({"손실": total_loss / num_batches})

    # 에포크 메트릭 계산 및 표시
    avg_loss = total_loss / num_batches
    test_acc = calculate_accuracy(model, tokenizer, test_loader)
    print(f"에포크 {epoch+1} - 평균 손실: {avg_loss:.4f}, 테스트 정확도: {test_acc:.4f}")

# 최종 모델 성능 계산
train_acc = calculate_accuracy(model, tokenizer, train_loader)
print(f"훈련 정확도: {train_acc:.4f}")
print(f"테스트 정확도: {test_acc:.4f}")

# 훈련된 모델과 토크나이저 저장
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

# 샘플 입력으로 모델 테스트
test_input = "I'm so happy to be able to finetune an LLM!"
test_model("./finetuned_model", test_input)

사용 중인 장치: cuda


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

에포크 1/2: 100%|██████████| 1125/1125 [01:19<00:00, 14.22it/s, 손실=0.119]


에포크 1 - 평균 손실: 0.1189, 테스트 정확도: 0.9320


에포크 2/2: 100%|██████████| 1125/1125 [01:18<00:00, 14.41it/s, 손실=0.0591]


에포크 2 - 평균 손실: 0.0591, 테스트 정확도: 0.9400
훈련 정확도: 0.9456
테스트 정확도: 0.9400
사용 중인 장치: cuda
입력: I'm so happy to be able to finetune an LLM!
생성된 감정: joy
